In [1]:
import datamanager
import torch
import pathlib

config = datamanager.HyFluidNeRFDataManagerConfig(dataparser=datamanager.HyFluidDataParserConfig())
config.dataparser.data = pathlib.Path("../data/ScalarReal")
manager: datamanager.HyFluidNeRFDataManager = config.setup(device=torch.device("cuda"))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Caching all 480 images.

Output()

Caching all 120 images.

Output()

In [8]:
ray_bundle, batch = manager.next_train()
print(ray_bundle)
print(batch['image'])
print(batch['indices'])

RayBundle(origins=tensor([[1.2956, 0.0244, 0.4975],
        [1.2956, 0.0244, 0.4975],
        [1.2956, 0.0244, 0.4975],
        ...,
        [1.2956, 0.0244, 0.4975],
        [1.2956, 0.0244, 0.4975],
        [1.2956, 0.0244, 0.4975]], device='cuda:0'), directions=tensor([[-0.7543,  0.0600, -0.6537],
        [-0.7328,  0.0286, -0.6799],
        [-0.7801,  0.4343, -0.4504],
        ...,
        [-0.7571,  0.1823, -0.6273],
        [-0.7956, -0.0186, -0.6055],
        [-0.7623,  0.0252, -0.6467]], device='cuda:0'), pixel_area=tensor([[1.3628e-07],
        [1.3267e-07],
        [1.3528e-07],
        ...,
        [1.4452e-07],
        [1.2818e-07],
        [1.3305e-07]], device='cuda:0'), camera_indices=tensor([[393],
        [451],
        [415],
        ...,
        [466],
        [399],
        [375]], device='cuda:0'), nears=None, fars=None, metadata={'directions_norm': tensor([[1.0240],
        [1.0333],
        [1.0267],
        ...,
        [1.0043],
        [1.0452],
        [1.032

In [18]:
from nerfstudio.model_components.ray_samplers import UniformSampler
from nerfstudio.model_components.scene_colliders import NearFarCollider
from nerfstudio.cameras.rays import RaySamples

collider = NearFarCollider(near_plane=1.1, far_plane=1.5)
sampler_uniform = UniformSampler(num_samples=192)

collider(ray_bundle)
ray_samples_uniform : RaySamples = sampler_uniform(ray_bundle)

print(ray_samples_uniform)

RaySamples(frustums=Frustums(origins=tensor([[[1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975],
         ...,
         [1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975]],

        [[1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975],
         ...,
         [1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975]],

        [[1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975],
         ...,
         [1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975]],

        ...,

        [[1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975],
         ...,
         [1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975],
         [1.2956, 0.0244, 0.4975]],

        [[1.2956, 0.0244, 0.4975],
         [

In [17]:
positions = ray_samples_uniform.frustums.get_positions()

print(positions.shape)

torch.Size([1024, 192, 3])
